In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rahul\\Downloads\\Facial_review_system'

In [5]:
#this is for entity
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Facial_Review_System.constants import *
from Facial_Review_System.utils.common import read_yaml, create_directories

In [9]:
#this is for config
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [10]:
import os
import zipfile
import gdown
from Facial_Review_System import logger
from Facial_Review_System.utils.common import get_size

In [11]:
#this is for components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/prepare_base_model", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [12]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-29 00:36:35,873: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-29 00:36:35,879: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-29 00:36:35,881: INFO: common: created directory at: artifacts]
[2024-04-29 00:36:35,883: INFO: common: created directory at: artifacts/data_ingestion]
[2024-04-29 00:36:35,884: INFO: 3905061188: Downloading data from https://drive.google.com/file/d/11xJiUrihSSiFMxvm9M_KVur52cf0fztW/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


KeyboardInterrupt: 